In [15]:
import ipynb.fs #Boilerplate for import ipynb.fs #Boilerplate for ipynb botebooks as modules

from utils import facenet
from utils import lfw
from .defs.Facenet_impl import get_face_embeddings
import os, sys
import numpy as np
from sklearn import metrics
from scipy.optimize import brentq
from scipy import interpolate

In [9]:
# Embeddings are calculated using the pairs from http://vis-www.cs.umass.edu/lfw/pairs.txt
pairs_path = '/home/prudhvi/Desktop/Distillation/mobilenet-facenet-distillation/data/pairs.txt'
pairs = lfw.read_pairs(pairs_path)

In [10]:
dataset_path = "~/Desktop/Datasets/lfw/lfw_mtcnnpy_160"
# Get the paths for the corresponding images
paths, actual_issame = lfw.get_paths(os.path.expanduser(dataset_path), pairs)

In [11]:
embeddings = get_face_embeddings(paths, mode = "paths")

Loading feature extraction model
Model directory: /home/prudhvi/Desktop/Models/tf/Facenet/20180402-114759
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
INFO:tensorflow:Restoring parameters from /home/prudhvi/Desktop/Models/tf/Facenet/20180402-114759/model-20180402-114759.ckpt-275
embedding size 512
Calculating features for images
Number of epochs 188


In [16]:
tpr, fpr, accuracy, val, val_std, far = lfw.evaluate(embeddings, actual_issame, nrof_folds=10, distance_metric=1, subtract_mean=1)
    
print('Accuracy: %2.5f+-%2.5f' % (np.mean(accuracy), np.std(accuracy)))
print('Validation rate: %2.5f+-%2.5f @ FAR=%2.5f' % (val, val_std, far))
    
auc = metrics.auc(fpr, tpr)
print('Area Under Curve (AUC): %1.3f' % auc)
eer = brentq(lambda x: 1. - x - interpolate.interp1d(fpr, tpr)(x), 0., 1.)
print('Equal Error Rate (EER): %1.3f' % eer)

Accuracy: 0.98833+-0.00441
Validation rate: 0.89667+-0.02494 @ FAR=0.00067
Area Under Curve (AUC): 0.999
Equal Error Rate (EER): 0.012
